# Database manipulation with polars

You can use any database library to manipulate data in the ARIEL database. In this example we will be using polars.

In [ ]:
# Import the libraries
import polars as pl
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

## 1. How to get data about the experiment

As explained above, ARIEL stores every individual that existed during evolution in an SQL database. This includes individuals that died early as well as those that survived until the end. By querying this database, we can recreate population data and extract statistics about the evolutionary process.

### 1.1 Loading the database into a Polars DataFrame

All data about individuals is stored in the table individual.
Below is how to load this table into a Polars DataFrame.

In [ ]:
# Load database table into Polars
file_path = "__data__/run_0033"

conn = sqlite3.connect(f"{file_path}/database.db")

data = pl.DataFrame(
    conn.execute("SELECT * FROM individual").fetchall(),
    schema=[col[1] for col in conn.execute("PRAGMA table_info(individual)")]
)

data.head()
data.tail()

### 1.2 Reconstructing the population per generation

ARIEL does not store a list of individuals per generation. Instead, each individual records:
- time_of_birth: the generation it was created
- time_of_death: the generation it disappeared

We reconstruct the population for every generation using this information.

In [ ]:
# Determine generation range
min_gen = int(data["time_of_birth"].min())
max_gen = int(data["time_of_death"].max())

# Collect individuals alive per generation
population_per_gen = {
    gen: data.filter(
        (pl.col("time_of_birth") <= gen) & (pl.col("time_of_death") >= gen) )["id"].to_list() # TODO message john bug fix
    for gen in range(min_gen, max_gen + 1)
    }

# Structure data as a Polars dataframe
pop_df = pl.DataFrame({
    "generation": list(population_per_gen.keys()),
    "individuals": list(population_per_gen.values()),
    "pop size": [len(v) for v in population_per_gen.values()]
    })

pop_df.head()
# pop_df.tail()


In [ ]:
# last_gen = data["time_of_birth"].max()       
# last_gen_df = data.filter(pl.col("time_of_birth") == last_gen).select(["id", "fitness_", "tags_"])
# print(f"Last generation: {last_gen}")                                                                                                             
# print(f"Population size: {len(last_gen_df)}")                                                                                                      

## 2. Computing fitness statistics per generation

Now that we know which individuals were alive in each generation, we can compute:
- Mean fitness
- Standard deviation
- Best (minimum) fitness

In [ ]:
# Extract fitness values indexed by individual id
fitness_by_id = data.select(["id", "fitness_"])

means, stds, bests = [], [], []

for gen in pop_df["generation"]:
    ids = population_per_gen.get(gen, [])
    if not ids:
        means.append(np.nan)
        stds.append(np.nan)
        bests.append(np.nan)
        continue

    fits = (
        fitness_by_id
        .filter(pl.col("id").is_in(ids))
        .select(pl.col("fitness_").cast(pl.Float64))
        .to_series()
        .to_numpy()
    )

    if fits.size == 0:
        means.append(np.nan)
        stds.append(np.nan)
        bests.append(np.nan)
    else:
        means.append(float(np.mean(fits)))
        stds.append(float(np.std(fits)))
        bests.append(float(np.max(fits)))

# Add the statistics to the dataframe
pop_df = pop_df.with_columns([
    pl.Series("fitness_mean", means),
    pl.Series("fitness_std", stds),
    pl.Series("fitness_best", bests)
])

pop_df.tail()


## 3. Plotting the fitness progression

With the computed statistics, we can visualize how fitness changes over generations.

In [ ]:
df = pop_df.drop_nulls(subset=["fitness_mean"])

x = df["generation"]
mean = df["fitness_mean"]
std = df["fitness_std"]
best = df["fitness_best"]

plt.figure(figsize=(10, 5))
plt.plot(x, mean, label="Fitness mean", linewidth=2)
plt.plot(x, best, "--", label="Fitness best", linewidth=2)
plt.fill_between(x, mean - std, mean + std, alpha=0.25, label="Mean ± Std")

plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Fitness statistics per generation")
plt.legend()
# plt.yticks(range(0, int(max(df["fitness_mean"])), 2))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
import json
ctk_strings = [json.loads(t).get('ctk_string') for t in data['tags_'][-50:]]
print(ctk_strings)

In [ ]:
import canonical_toolkit as ctk

In [ ]:
ctk.quick_view(ctk.node_from_string('C[f(H6B2[r(B7H5B[l(B4[r(B6BH4B)])])])]<b(B5B7H5H7)>').to_graph())

In [ ]:
# raise UserWarning

In [ ]:
ctk.snapshot_saver([ctk.node_from_string(ctk_string).to_graph() for ctk_string in ctk_strings], save_folder=f"{file_path}/snapshots")

In [ ]:
from canonical_toolkit.morphology.visual.plots import SnapshotGridPlotter

plotter = SnapshotGridPlotter.from_snapshot_folder(f"{file_path}/snapshots", grid_shape=(10, 10), snapshots_per_subplot=1)
plotter.plot()


In [ ]:
last_gen = data["time_of_death"].max()                                                                                                            
                                                                                                                                                    
# Survivors are those whose death time equals the final generation                                                                                
# AND they weren't just born to die immediately                                                                                                   
survivors = data.filter(                                                                                                                          
    (pl.col("time_of_death") == last_gen) &                                                                                                       
    (pl.col("time_of_birth") < last_gen)  # survived at least one generation                                                                      
).select(["id", "fitness_", "time_of_birth", "time_of_death"])                                                                                    
                                                                                                                                                
print(f"Survivor count: {len(survivors)}")                                                                                                                                                                                                                                                 
                                                                                                                                                
                                                                                                                                                
last_gen = data["time_of_death"].max()                                                                                                            
                                                                                                                                                
# Break down by birth time                                                                                                                        
breakdown = data.filter(                                                                                                                          
    pl.col("time_of_death") >= last_gen                                                                                                           
).group_by("time_of_birth").agg(pl.count())                                                                                                       
                                                                                                                                                
print(breakdown.sort("time_of_birth"))


In [ ]:
survivors = data.filter(                                                                                                                          
    (pl.col("time_of_death") == last_gen) &                                                                                                       
    (pl.col("time_of_birth") < last_gen)                                                                                                          
)                                                                                                                                                 
                                                                                                                                                
print(f"Fitness stats of survivors:")                                                                                                             
print(f"  Mean: {survivors['fitness_'].mean()}")                                                                                                  
print(f"  Std:  {survivors['fitness_'].std()}")                                                                                                   
print(f"  Max:  {survivors['fitness_'].max()}")                                                                                                   
print(f"  Min:  {survivors['fitness_'].min()}")                                                                                                   
                                                                                                                                                
# Full dataframe                                                                                                                                  
survivors.select(["id", "fitness_", "time_of_birth"])  

In [ ]:
gen_stats = []                                                                                                                                    
max_gen = int(data["time_of_death"].max())                                                                                                        
                                                                                                                                                
for gen in range(0, max_gen + 1):                                                                                                                 
    # Individuals alive DURING this generation (born <= gen, died > gen)                                                                          
    alive_during = data.filter(                                                                                                                   
        (pl.col("time_of_birth") <= gen) & (pl.col("time_of_death") > gen)                                                                        
    )                                                                                                                                             
                                                                                                                                                
    # Individuals born this generation                                                                                                            
    born_this_gen = data.filter(pl.col("time_of_birth") == gen)                                                                                   
                                                                                                                                                
    # Individuals that died this generation                                                                                                       
    died_this_gen = data.filter(pl.col("time_of_death") == gen)                                                                                   
                                                                                                                                                
    gen_stats.append({                                                                                                                            
        "generation": gen,                                                                                                                        
        "alive_during": len(alive_during),                                                                                                        
        "born": len(born_this_gen),                                                                                                               
        "died": len(died_this_gen),                                                                                                               
    })                                                                                                                                            
                                                                                                                                                
gen_df = pl.DataFrame(gen_stats)                                                                                                                  
gen_df                  

In [ ]:
import json                                                                                                                                       
                                                                                                                                                    
# Get top X fittest individuals (sorted by fitness descending)                                                                                    
top_x = 10  # change this to your desired number                                                                                                  
                                                                                                                                                
top_individuals = data.sort("fitness_", descending=True).head(top_x)                                                                              
                                                                                                                                                
# Extract CTK strings                                                                                                                             
for row in top_individuals.iter_rows(named=True):                                                                                                 
    ctk_string = json.loads(row['tags_']).get('ctk_string')                                                                                       
    print(f"ID: {row['id']}, Fitness: {row['fitness_']:.6f}, CTK: {ctk_string}")                                                                  
                                                                                                                                                
                                                                                                                                                
top_x = 10                                                                                                                                        
top_ctk_strings = [                                                                                                                               
    json.loads(t).get('ctk_string')                                                                                                               
    for t in data.sort("fitness_", descending=True).head(top_x)['tags_']                                                                          
]                                                                                                                                                 
print(top_ctk_strings)